In [1]:
%pip install osmium folium

Note: you may need to restart the kernel to use updated packages.


In [2]:
import vrp
import geographs
import folium

In [3]:
Depot_coord = (18.463521, -69.934688)#(18.481891, -69.913954)8299470296
qty_clients = 40
cap_trucks = 100
truck_vel = 60

In [4]:
print("Loading data... ")
geoGraph = geographs.GeoGraph(limit_coords=Depot_coord)
print("Data loaded! ")

Loading data... 
Available nodes before condition: 133161
Available nodes after condition: 12573
Data loaded! 


In [5]:
Depot_node, clients, qty_trucks = vrp.create_nodes(geoGraph, Depot_coord, qty_clients, cap_trucks)

print("Depot node: ", Depot_node)
print("qty of trucks: ", qty_trucks)

Depot node:  GeoNode 309426928 at pos (-699345419, 184633062), coords (18.4633062, -69.9345419)
qty of trucks:  4


In [15]:
colors = ['red', 'blue', 'green', 'purple', 'darkblue']
depot_color = 'black'

map = folium.Map(location=Depot_coord, zoom_start=17)

for client in clients:
    client_node_marker = [client.node.lat, client.node.lon]
    folium.Marker(client_node_marker,popup=client.node.name).add_to(map)

depot_marker = [Depot_node.lat, Depot_node.lon]
folium.Marker(depot_marker,popup=Depot_node.name, icon=folium.Icon(color=depot_color)).add_to(map)

map

In [6]:
qty_poblacion = 200
n_elite = 10
n_generations = 50
prob_de_mut = 0.5

create_clusters = vrp.create_clusters(geoGraph, qty_clients, qty_trucks, cap_trucks, qty_poblacion, n_elite, n_generations, clients, prob_de_mut)
solution = create_clusters.create_clusters()
print("solution: ", solution)

solution:  [0, 3, 2, 1, 2, 2, 2, 3, 1, 0, 2, 3, 2, 0, 2, 3, 1, 1, 1, 1, 3, 2, 3, 0, 0, 3, 0, 1, 3, 0, 2, 3, 1, 2, 0, 1, 3, 1, 0, 0]


In [18]:
colors = ['red', 'blue', 'green', 'purple', 'darkblue']
depot_color = 'black'

map = folium.Map(location=Depot_coord, zoom_start=17)

for truck_id in range(qty_trucks):
    clients_id = vrp.find_indices(solution, truck_id)
    for client_id in clients_id:
        client = clients[client_id]
        client_node_marker = [client.node.lat, client.node.lon]
        folium.Marker(client_node_marker,popup=client.node.name, icon=folium.Icon(color=colors[truck_id])).add_to(map)

depot_marker = [Depot_node.lat, Depot_node.lon]
folium.Marker(depot_marker,popup=Depot_node.name, icon=folium.Icon(color=depot_color)).add_to(map)

map

In [7]:
n_nearest_nodes = 4

routes, paths = vrp.create_routes(geoGraph, Depot_node, clients, qty_trucks, solution, n_nearest_nodes)

print("routes: ", routes)

for graph in routes:
    print(graph.get_nodes())


Truck 1/4
Truck 2/4
Truck 3/4
Truck 4/4
routes:  [<graphs.Graph object at 0x0000014D3B7CBB20>, <graphs.Graph object at 0x0000014D09592C10>, <graphs.Graph object at 0x0000014D3B7CBAF0>, <graphs.Graph object at 0x0000014D3B7CBCD0>]
{<geographs.GeoNode object at 0x0000014D2B536610>, <geographs.GeoNode object at 0x0000014D2B536C40>, <geographs.GeoNode object at 0x0000014D33D23C70>, <geographs.GeoNode object at 0x0000014D2B064280>, <geographs.GeoNode object at 0x0000014D30AC70A0>, <geographs.GeoNode object at 0x0000014D2B124EB0>, <geographs.GeoNode object at 0x0000014D2B100F10>, <geographs.GeoNode object at 0x0000014D2B512B20>, <geographs.GeoNode object at 0x0000014D2BE38550>, <geographs.GeoNode object at 0x0000014D33D0B9A0>}
{<geographs.GeoNode object at 0x0000014D2C24BA00>, <geographs.GeoNode object at 0x0000014D37964220>, <geographs.GeoNode object at 0x0000014D2B60E640>, <geographs.GeoNode object at 0x0000014D2B064280>, <geographs.GeoNode object at 0x0000014D2B80A8B0>, <geographs.GeoNode

In [11]:
colors = ['red', 'blue', 'green', 'purple', 'darkblue']
depot_color = 'black'

map = folium.Map(location=Depot_coord, zoom_start=17)

i = 0
for route in paths:
    for initial_node_name in route:
        for goal_node_name in route[initial_node_name]:
            
            initial_node = routes[i].get_node_by_name(initial_node_name)
            goal_node = routes[i].get_node_by_name(goal_node_name)

            initial_marker = [initial_node.lat, initial_node.lon]
            folium.Marker(initial_marker,popup=initial_node.name, icon=folium.Icon(color=colors[i])).add_to(map)
            
            goal_marker = [goal_node.lat, goal_node.lon]
            folium.Marker(goal_marker,popup=goal_node.name, icon=folium.Icon(color=colors[i])).add_to(map)

            path = route[initial_node_name][goal_node_name]
            points = []

            for node in path:
                point = [node.lat, node.lon]
                points.append(point)

            folium.PolyLine(points, color=colors[i], weight=5, opacity=1).add_to(map)
    i+=1

depot_marker = [Depot_node.lat, Depot_node.lon]
folium.Marker(depot_marker,popup=Depot_node.name, icon=folium.Icon(color=depot_color)).add_to(map)

map

In [12]:
import genetic_algorithms_vin as gav
import random

answers = []

for route in routes:

    SUCCESS = False

    while not SUCCESS:

        graph = route.get_adj_matrix(Depot_node)

        #Params
        num_gen = 5000
        population_num = 50
        solutions_to_choose = 5
        mutation_prob = 0.7

        cities = list(range(0,len(graph)))

        initial_city = 0

        #initial population
        solutions = gav.create_population(population_num, cities, initial_city)
        #print('Propulation: ', solutions)

        best_solutions_by_gen = []

        #For each generation
        for gen in range(num_gen):

            #Calculate the total distance of each solution
            distances = gav.calc_distances_paths(graph, initial_city, solutions)
            #print("distances: ", distances)
            
            #Calculate the fitness for each solution
            fitnesess = gav.calc_fitness(distances)
            #print("fitnesess: ", fitnesess)

            if(sum(fitnesess) == 0):
                choosen_solutions = solutions[:solutions_to_choose]
            else:
                #Choose k solutions randomly based on their fitnesess
                choosen_solutions = random.choices(solutions, weights=fitnesess, k=solutions_to_choose)
            #print("choosen solutions", choosen_solutions)

            #Save the best k solutions of this generation
            best_solutions_by_gen.append(choosen_solutions)

            #Use best solutions to create a new generation by croosover and mutation (based in mutation_prob)
            crossovered_solutions = gav.crossover_solutions(choosen_solutions, population_num)
            #print("Crossovered_solutions: ", crossovered_solutions)
            solutions = gav.mutate_solutions(crossovered_solutions, mutation_prob)
            #print("Mutated_solutions: ", solutions)

        best_solution_by_gen = []

        for gen_solutions in best_solutions_by_gen:
            distances = gav.calc_distances_paths(graph, initial_city, gen_solutions)
            best_solution_of_gen = gav.sort_solutions_by_distance(gen_solutions, distances)[-1]
            best_solution_by_gen.append(best_solution_of_gen)

        final_answer, cost = sorted(best_solution_by_gen,key=lambda x: x[1])[0]

        final_answer.append(initial_city)

        print("Answer: ", final_answer, cost)

        SUCCESS = cost != float('inf')

    answers.append(final_answer)

Answer:  [0, 8, 4, 7, 1, 2, 6, 9, 3, 5, 0] 23.07064508907405
Answer:  [0, 4, 7, 5, 3, 1, 10, 8, 2, 9, 6, 0] 22.969617182025605
Answer:  [0, 2, 6, 4, 5, 10, 3, 9, 8, 7, 1, 0] 24.155119637301944
Answer:  [0, 1, 6, 10, 4, 2, 8, 5, 3, 9, 7, 0] 24.381167439700572


In [13]:
colors = ['red', 'blue', 'green', 'purple', 'darkblue']
depot_color = 'black'

map = folium.Map(location=Depot_coord, zoom_start=17)

i = 0
for route_graph,route, answer in zip(routes,paths, answers):
    
    nodes_answers = [route_graph.index_to_node[answer_node] for answer_node in answer]

    #print("Nodes answers: ", nodes_answers)

    j = 0
    for j in range(len(nodes_answers)-1):
        initial_node = routes[i].get_node_by_name(nodes_answers[j])
        goal_node = routes[i].get_node_by_name(nodes_answers[j+1])

        initial_marker = [initial_node.lat, initial_node.lon]
        folium.Marker(initial_marker,popup=initial_node.name, icon=folium.Icon(color=colors[i])).add_to(map)
        
        goal_marker = [goal_node.lat, goal_node.lon]
        folium.Marker(goal_marker,popup=goal_node.name, icon=folium.Icon(color=colors[i])).add_to(map)

        #print("Initial node: ", initial_node)
        #print("goal node: ", goal_node)

        path = route[initial_node.name][goal_node.name]
        points = []

        for node in path:
            point = [node.lat, node.lon]
            points.append(point)

        folium.PolyLine(points, color=colors[i], weight=5, opacity=1).add_to(map)
    i+=1

depot_marker = [Depot_node.lat, Depot_node.lon]
folium.Marker(depot_marker,popup=Depot_node.name, icon=folium.Icon(color=depot_color)).add_to(map)

map

In [20]:
colors = ['red', 'blue', 'green', 'purple', 'darkblue']
depot_color = 'black'

map = folium.Map(location=Depot_coord, zoom_start=17)

i = 2
route_graph,route, answer = routes[i],paths[i], answers[i]
nodes_answers = [route_graph.index_to_node[answer_node] for answer_node in answer]

#print("Nodes answers: ", nodes_answers)

j = 0
for j in range(len(nodes_answers)-1):
    initial_node = routes[i].get_node_by_name(nodes_answers[j])
    goal_node = routes[i].get_node_by_name(nodes_answers[j+1])

    initial_marker = [initial_node.lat, initial_node.lon]
    folium.Marker(initial_marker,popup=initial_node.name, icon=folium.Icon(color=colors[i])).add_to(map)
    
    goal_marker = [goal_node.lat, goal_node.lon]
    folium.Marker(goal_marker,popup=goal_node.name, icon=folium.Icon(color=colors[i])).add_to(map)

    #print("Initial node: ", initial_node)
    #print("goal node: ", goal_node)

    path = route[initial_node.name][goal_node.name]
    points = []

    for node in path:
        point = [node.lat, node.lon]
        points.append(point)

    folium.PolyLine(points, color=colors[i], weight=5, opacity=1).add_to(map)

depot_marker = [Depot_node.lat, Depot_node.lon]
folium.Marker(depot_marker,popup=Depot_node.name, icon=folium.Icon(color=depot_color)).add_to(map)

map